In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy.stats import spearmanr
from scipy.stats import norm

import statsmodels.api as sm
from datetime import datetime
# from bisect import bisect_left, bisect_right
import lmom

Calculate the return period discharge (Qt) for each of the rivers

In [ ]:
c = 68
op = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HIDnew\05_HYDzoom" + r"\C%d_" %c + r"disc1d_"
    
AM1 = pd.read_pickle(op + "AM1.pkl")
AM2 = pd.read_pickle(op + "AM2.pkl")
AM3 = pd.read_pickle(op + "AM3.pkl")

In [ ]:
def ecdf(data):
    """ Compute Empirical CDF """
    x = np.sort(data)
    n = x.size
    rank = np.arange(1, n+1)
    y = rank / (n+1)
    return x, y

def marginalsPOT(b1, b2, set1):
    #Marginal distributions - Parameter estimation
    LMU1 = lmom.samlmu(set1[b1])
    LMU2 = lmom.samlmu(set1[b2])

    #General pareto fit
    gpa_fit1 = lmom.pelgpa(LMU1)
    gpa_fit2 = lmom.pelgpa(LMU2)
    
    rho, Sp = spearmanr(set1[b1], set1[b2])
    #rho = set1[b1].corr(set1[b2])
    return gpa_fit1, gpa_fit2, rho

def marginalsAM(b1, b2, set1):
    #Marginal distributions - Parameter estimation
    LMU1 = lmom.samlmu(set1[b1])
    LMU2 = lmom.samlmu(set1[b2])

    #General pareto fit
    gev_fit1 = lmom.pelgev(LMU1)
    gev_fit2 = lmom.pelgev(LMU2)
    
    rho, Sp = spearmanr(set1[b1], set1[b2])
    #rho = set1[b1].corr(set1[b2])
    return gev_fit1, gev_fit2, rho

In [ ]:
def empinv(yp, xed, yed):
    xp = []
    for i in range(len(yed)):
        if yp < yed[0]:
            xp=xed[0]
            break
        elif yp > yed[len(yed)-1]:
            xp=(xed[len(yed)-1])
            break
        elif yp == yed[i-1]:
            xp=(xed[i-1])
        elif yed[i-1] < yp < yed[i]:
            y1 = yed[i-1]
            y2 = yed[i]
            x1 = xed[i-1]
            x2 = xed[i]
            xcal = x1 + ((x2-x1)/(y2-y1)) * (yp-y1)
            xp=(xcal)
        #else:
         #   xp.append(xed[len(yed)-1])
    return xp

In [ ]:
QS1 = AM1['Q_S1'].values
QS2 = AM2['Q_S2'].values

In [ ]:
x, y = ecdf(QS1)
T = 2
p = 1 - 1/T
QS1_T2 = empinv(p, x, y)
QS1_T2
plt.plot(x,y)
print(QS1_T2)

In [ ]:
x, y = ecdf(QS2)
T = 2
p = 1 - 1/T
QS2_T2 = empinv(p, x, y)
QS2_T2
plt.plot(x,y)
print(QS2_T2)

In [ ]:
def Q100tradAM(b1, b2, Tc_1):
    s1, s2, am1 = dar.AMconf(b1, b2)
    
    #Determine the distribution parameters
    LMU1 = lmom.samlmu(am1['Qc']) #q1+q2 check
    gev_fit1 = lmom.pelgev(LMU1)

    #determine Q100
    T=100
    q = 1 - 1/T
    Q= lmom.quagev(q, gev_fit1)
    
    #N-samples
    n50=randSet1AM(b1, b2, am1, 50)
    LMU1 = lmom.samlmu(n50['Qc']) #q1+q2 check
    gev_fit1 = lmom.pelgev(LMU1)
    Q50= lmom.quagev(q, gev_fit1)
    
    n100=randSet1AM(b1, b2, am1, 100)
    LMU1 = lmom.samlmu(n100['Qc']) #q1+q2 check
    gev_fit1 = lmom.pelgev(LMU1)
    Q100= lmom.quagev(q, gev_fit1)
    
    n500=randSet1AM(b1, b2, am1, 500)
    LMU1 = lmom.samlmu(n500['Qc']) #q1+q2 check
    gev_fit1 = lmom.pelgev(LMU1)
    Q500= lmom.quagev(q, gev_fit1)
    
    n1000=randSet1AM(b1, b2, am1, 1000)
    #Determine the distribution parameters
    LMU1 = lmom.samlmu(n1000['Qc']) #q1+q2 check
    gev_fit1 = lmom.pelgev(LMU1)
    #determine Q100
    Q1000= lmom.quagev(q, gev_fit1)

    
    return Q, Q50, Q100, Q500, Q1000